In [1]:
#!/usr/bin/env python

import torch
import torch.nn as nn
import pickle as pk
from sklearn.model_selection import KFold
from config import config
from function import *


import warnings
warnings.filterwarnings('once')

words, sentences, labels, tags_vals, tag2idx, add_len = read_data(config, "data/dataset.csv") #data_path = "data/dataset.csv"
config['num_labels'] = len(tag2idx)
data_fold = vectorization(config, sentences, labels, tags_vals, tag2idx, add_len)

config['device'] = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print('The device is ' + torch.cuda.get_device_name(0)) 
print('The number of device is ', n_gpu) 





Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


Using TensorFlow backend.


The longest sentence has 697 tokens.
The device is GeForce RTX 2080 Ti
The number of device is  1


In [2]:
def cross_validation1(config):
    print('Cross Validation')
    fold_num = config['fold_num']
    kfold = KFold(fold_num, shuffle=True, random_state=None)
    # enumerate splits
    f = 1
    eval_accuracy_fold = [] 
    f1_fold = []
    recall_fold = []
    for train_index, test_index in kfold.split(data_fold[0]):
        dataloader, count = myDataLoader(config, data_fold, train_index, test_index)

    #             weight = torch.tensor(np.log(max(count)/count)+config['mu'])
        weight = None
        print('')
        print('Fold {}:'.format(f))
        model = BuildModel(config, weight)
        _, eval_accuracy, f1, precision, recall = train(config, model, dataloader, if_plot=False, fold_id=f)
        eval_accuracy_fold.append(eval_accuracy)
        f1_fold.append(f1)
        recall_fold.append(recall)
        f+=1
        model = None
        
    ave_acc = sum(eval_accuracy_fold)/fold_num
    ave_f1 = sum(f1_fold)/fold_num
    ave_recall = sum(recall_fold)/fold_num
    
    print('accuracy of folds: {}'.format(eval_accuracy_fold))
    print('average accuracy: {}'.format(ave_acc))
    print('f1 of folds: {}'.format(f1_fold))
    print('average f1: {}'.format(ave_f1))
    print('recall of folds: {}'.format(recall_fold))
    print('average recall: {}'.format(ave_recall))

    return ave_acc, ave_f1, ave_recall


In [4]:
c = config.copy()
lr = []
acc = []
f1 = []
recall = []
for r in range(1,11):
    c['lr']=r*10**(-5)
    print('lr=',c['lr'])
    lr.append(c['lr'])
    ave_acc, ave_f1, ave_recall = cross_validation1(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
lr_list = list(zip(lr,acc,f1,recall))

lr= 1e-05
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9748396645288604, Macro Fbeta-Score: 0.826982702674196, Macro Precision: 0.844291666927801, Macro Recall: 0.8372857035283944

Fold 2:
The best result: 
Validation Accuracy: 0.9708920187793427, Macro Fbeta-Score: 0.9156477025051841, Macro Precision: 0.8755980934979773, Macro Recall: 0.9306508339221232

Fold 3:
The best result: 
Validation Accuracy: 0.9819455672325519, Macro Fbeta-Score: 0.9341724949214633, Macro Precision: 0.9535135984650771, Macro Recall: 0.9317384648078311

Fold 4:
The best result: 
Validation Accuracy: 0.9793267956533263, Macro Fbeta-Score: 0.8857836694705032, Macro Precision: 0.8647096534326006, Macro Recall: 0.8938511001856614

Fold 5:
The best result: 
Validation Accuracy: 0.979917469050894, Macro Fbeta-Score: 0.9136657830753225, Macro Precision: 0.8724618951218339, Macro Recall: 0.932556835446099
accuracy of folds: [0.9748396645288604, 0.9708920187793427, 0.9819455672325519, 0.9793267956

/home/junliang/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/junliang/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


The best result: 
Validation Accuracy: 0.9894498869630746, Macro Fbeta-Score: 0.9657737164171141, Macro Precision: 0.9416109759542596, Macro Recall: 0.9737946295639205
accuracy of folds: [0.972463408583478, 0.9792833146696529, 0.9727636181909045, 0.9628281778008838, 0.9894498869630746]
average accuracy: 0.9753576812415987
f1 of folds: [0.8746998265117814, 0.9000129227823768, 0.8890562454168303, 0.8674031748886312, 0.9657737164171141]
average f1: 0.8993891772033468
recall of folds: [0.883228978065454, 0.8958305817010239, 0.9256608861603018, 0.8903710948482346, 0.9737946295639205]
average recall: 0.913777234067787
lr= 4e-05
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9708966963817515, Macro Fbeta-Score: 0.8450235801587846, Macro Precision: 0.8117390540755677, Macro Recall: 0.8604498052705044

Fold 2:
The best result: 
Validation Accuracy: 0.9808439755691283, Macro Fbeta-Score: 0.9400610846839131, Macro Precision: 0.8640961428943209, Macro Recall: 0.9659053505498445

The best result: 
Validation Accuracy: 0.9670542635658915, Macro Fbeta-Score: 0.8716756133992629, Macro Precision: 0.8191846043607304, Macro Recall: 0.894626207618036
accuracy of folds: [0.982740021574973, 0.9846566000974184, 0.9820359281437125, 0.9821849620120513, 0.9670542635658915]
average accuracy: 0.9797343550788093
f1 of folds: [0.9168344933600772, 0.9573640333259475, 0.9408353346852726, 0.8716584015917257, 0.8716756133992629]
average f1: 0.9116735752724573
recall of folds: [0.9321464610091907, 0.9800295395700758, 0.9493097906876268, 0.8752588512179306, 0.894626207618036]
average recall: 0.9262741700205719
lr= 0.0001
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9732441471571907, Macro Fbeta-Score: 0.8249816322270753, Macro Precision: 0.8279797472023445, Macro Recall: 0.8582666828464575

Fold 2:
The best result: 
Validation Accuracy: 0.9744049142564628, Macro Fbeta-Score: 0.886471706072969, Macro Precision: 0.8478348850886499, Macro Recall: 0.9058499074507094

In [5]:
lr_list_ = np.array(lr_list)
best_lr = lr_list_[np.argmax(lr_list_[:,3]),0]
print(best_lr)

8e-05


In [6]:
c = config.copy()
c['lr'] = best_lr
decay = []
acc = []
f1 = []
recall = []
for r in range(1,11):
    c['decay']=10**(-r)
    print('decay=',c['decay'])
    decay.append(c['decay'])
    ave_acc, ave_f1, ave_recall = cross_validation1(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
decay_list = list(zip(decay,acc,f1,recall))

decay= 0.1
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9653534506845488, Macro Fbeta-Score: 0.8448523087335491, Macro Precision: 0.8111545916590818, Macro Recall: 0.8668412797356507

Fold 2:
The best result: 
Validation Accuracy: 0.9848167539267015, Macro Fbeta-Score: 0.9551819375292362, Macro Precision: 0.9363653833215594, Macro Recall: 0.9608163983403104

Fold 3:
The best result: 
Validation Accuracy: 0.9874718115760461, Macro Fbeta-Score: 0.9671346382020549, Macro Precision: 0.9441014193677101, Macro Recall: 0.9748063934838916

Fold 4:
The best result: 
Validation Accuracy: 0.9744258872651357, Macro Fbeta-Score: 0.9088973598874366, Macro Precision: 0.8762293257958695, Macro Recall: 0.9296976771744634

Fold 5:
The best result: 
Validation Accuracy: 0.9800522441225362, Macro Fbeta-Score: 0.8938760824892962, Macro Precision: 0.9069937906706635, Macro Recall: 0.8982939180986287
accuracy of folds: [0.9653534506845488, 0.9848167539267015, 0.9874718115760461, 0.97442

The best result: 
Validation Accuracy: 0.9808850484420005, Macro Fbeta-Score: 0.9269503756461025, Macro Precision: 0.8648586369750486, Macro Recall: 0.9516610415038348

Fold 2:
The best result: 
Validation Accuracy: 0.97343306680423, Macro Fbeta-Score: 0.8303889057813335, Macro Precision: 0.7867654449461602, Macro Recall: 0.8489938031390509

Fold 3:
The best result: 
Validation Accuracy: 0.9736154649947754, Macro Fbeta-Score: 0.8644708537421977, Macro Precision: 0.8930094388620274, Macro Recall: 0.870174768973781

Fold 4:
The best result: 
Validation Accuracy: 0.9766846015885217, Macro Fbeta-Score: 0.9367863463113706, Macro Precision: 0.8821980860924642, Macro Recall: 0.9542406641765531

Fold 5:
The best result: 
Validation Accuracy: 0.9695456814777833, Macro Fbeta-Score: 0.8592482721813375, Macro Precision: 0.8495770776475992, Macro Recall: 0.8810692214664846
accuracy of folds: [0.9808850484420005, 0.97343306680423, 0.9736154649947754, 0.9766846015885217, 0.9695456814777833]
average a

In [7]:
decay_list_ = np.array(decay_list)
best_decay = decay_list_[np.argmax(decay_list_[:,3]),0]
print(best_decay)

0.001


In [8]:
def cross_validation2(config):
    print('Cross Validation')
    fold_num = config['fold_num']
    kfold = KFold(fold_num, shuffle=True, random_state=None)
    # enumerate splits
    f = 1
    eval_accuracy_fold = [] 
    f1_fold = []
    recall_fold = []
    for train_index, test_index in kfold.split(data_fold[0]):
        dataloader, count = myDataLoader(config, data_fold, train_index, test_index)

        weight = torch.tensor(np.log(max(count)/count)+config['mu'])
#         weight = None
        print('')
        print('Fold {}:'.format(f))
        model = BuildModel(config, weight)
        _, eval_accuracy, f1, recall = train(config, model, dataloader, if_plot=False, fold_id=f)
        eval_accuracy_fold.append(eval_accuracy)
        f1_fold.append(f1)
        recall_fold.append(recall)
        f+=1
        model = None
        
    ave_acc = sum(eval_accuracy_fold)/fold_num
    ave_f1 = sum(f1_fold)/fold_num
    ave_recall = sum(recall_fold)/fold_num
    
    print('accuracy of folds: {}'.format(eval_accuracy_fold))
    print('average accuracy: {}'.format(ave_acc))
    print('f1 of folds: {}'.format(f1_fold))
    print('average f1: {}'.format(ave_f1))
    print('recall of folds: {}'.format(recall_fold))
    print('average recall: {}'.format(ave_recall))

    return ave_acc, ave_f1, ave_recall


In [9]:
c = config.copy()
c['lr'] = best_lr
c['decay'] = best_decay
mu = []
acc = []
f1 = []
recall = []
for r in range(1,11):
    c['mu']=r
    print('mu=',c['mu'])
    mu.append(c['mu'])
    ave_acc, ave_f1, ave_recall = cross_validation2(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
mu_list = list(zip(mu,acc,f1,recall))

mu= 1
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9734280452512497, Macro Fbeta-Score: 0.8919614915761862, Macro Precision: 0.8482930895807321, Macro Recall: 0.9102463396387231


ValueError: too many values to unpack (expected 4)

In [ ]:
mu_list_ = np.array(mu_list)
best_mu = mu_list_[np.argmax(mu_list_[:,3]),0]
print(best_mu)

In [11]:
def cross_validation3(config):
    print('Cross Validation')
    fold_num = config['fold_num']
    kfold = KFold(fold_num, shuffle=True, random_state=None)
    # enumerate splits
    f = 1
    eval_accuracy_fold = [] 
    f1_fold = []
    recall_fold = []
    for train_index, test_index in kfold.split(data_fold[0]):
        dataloader, count = myDataLoader(config, data_fold, train_index, test_index)

        weight = torch.tensor(np.maximun(np.log(config['mu']*sum(count)/count),1))
#         weight = None
        print('')
        print('Fold {}:'.format(f))
        model = BuildModel(config, weight)
        _, eval_accuracy, f1, recall = train(config, model, dataloader, if_plot=False, fold_id=f)
        eval_accuracy_fold.append(eval_accuracy)
        f1_fold.append(f1)
        recall_fold.append(recall)
        f+=1
        model = None
        
    ave_acc = sum(eval_accuracy_fold)/fold_num
    ave_f1 = sum(f1_fold)/fold_num
    ave_recall = sum(recall_fold)/fold_num
    
    print('accuracy of folds: {}'.format(eval_accuracy_fold))
    print('average accuracy: {}'.format(ave_acc))
    print('f1 of folds: {}'.format(f1_fold))
    print('average f1: {}'.format(ave_f1))
    print('recall of folds: {}'.format(recall_fold))
    print('average recall: {}'.format(ave_recall))

    return ave_acc, ave_f1, ave_recall

In [12]:
c = config.copy()
c['lr'] = best_lr
c['decay'] = best_decay
mu = []
acc = []
f1 = []
recall = []
for r in range(1,11):
    c['mu']=r/10
    print('mu=',c['mu'])
    mu.append(c['mu'])
    ave_acc, ave_f1, ave_recall = cross_validation2(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
mu_list = list(zip(mu,acc,f1,recall))

mu= 0.1
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9654654654654654, Macro F1-Score: 0.8617223089944377, Macro Recall: 0.9539985146222806

Fold 2:
The best result: 
Validation Accuracy: 0.9667346938775511, Macro F1-Score: 0.8822394585069461, Macro Recall: 0.9655481238220993

Fold 3:
The best result: 
Validation Accuracy: 0.9731270358306189, Macro F1-Score: 0.903065124828551, Macro Recall: 0.9476278468190812

Fold 4:
The best result: 
Validation Accuracy: 0.9752136752136752, Macro F1-Score: 0.8049494919646653, Macro Recall: 0.8998685993822922

Fold 5:
The best result: 
Validation Accuracy: 0.9731644482610562, Macro F1-Score: 0.8711458493430922, Macro Recall: 0.9223410458132193
accuracy of folds: [0.9654654654654654, 0.9667346938775511, 0.9731270358306189, 0.9752136752136752, 0.9731644482610562]
average accuracy: 0.9707410637296734
f1 of folds: [0.8617223089944377, 0.8822394585069461, 0.903065124828551, 0.8049494919646653, 0.8711458493430922]
average f1: 0.8646244

The best result: 
Validation Accuracy: 0.9724310776942355, Macro F1-Score: 0.8021728221669423, Macro Recall: 0.8455534961104486

Fold 3:
The best result: 
Validation Accuracy: 0.9704513256425825, Macro F1-Score: 0.8101657326539905, Macro Recall: 0.9271528209624674

Fold 4:
The best result: 
Validation Accuracy: 0.9739784509046554, Macro F1-Score: 0.8336808922468624, Macro Recall: 0.8966417192189557

Fold 5:
The best result: 
Validation Accuracy: 0.9810640216411182, Macro F1-Score: 0.9142113774015981, Macro Recall: 0.9674564257976823
accuracy of folds: [0.9866019866019866, 0.9724310776942355, 0.9704513256425825, 0.9739784509046554, 0.9810640216411182]
average accuracy: 0.9769053724969157
f1 of folds: [0.9487544602499688, 0.8021728221669423, 0.8101657326539905, 0.8336808922468624, 0.9142113774015981]
average f1: 0.8617970569438723
recall of folds: [0.9814649832054498, 0.8455534961104486, 0.9271528209624674, 0.8966417192189557, 0.9674564257976823]
average recall: 0.9236538890590008
mu= 0.

In [13]:
mu_list_ = np.array(mu_list)
best_mu = mu_list_[np.argmax(mu_list_[:,3]),0]
print(best_mu)

0.4


In [18]:
def cross_validation3(config):
    print('Cross Validation')
    fold_num = config['fold_num']
    kfold = KFold(fold_num, shuffle=True, random_state=None)
    # enumerate splits
    f = 1
    eval_accuracy_fold = [] 
    f1_fold = []
    recall_fold = []
    for train_index, test_index in kfold.split(data_fold[0]):
        dataloader, count = myDataLoader(config, data_fold, train_index, test_index)

        weight = torch.tensor(np.power(sum(count)/count,1/config['mu']))
#         weight = None
        print('')
        print('Fold {}:'.format(f))
        model = BuildModel(config, weight)
        _, eval_accuracy, f1, recall = train(config, model, dataloader, if_plot=False, fold_id=f)
        eval_accuracy_fold.append(eval_accuracy)
        f1_fold.append(f1)
        recall_fold.append(recall)
        f+=1
        model = None
        
    ave_acc = sum(eval_accuracy_fold)/fold_num
    ave_f1 = sum(f1_fold)/fold_num
    ave_recall = sum(recall_fold)/fold_num
    
    print('accuracy of folds: {}'.format(eval_accuracy_fold))
    print('average accuracy: {}'.format(ave_acc))
    print('f1 of folds: {}'.format(f1_fold))
    print('average f1: {}'.format(ave_f1))
    print('recall of folds: {}'.format(recall_fold))
    print('average recall: {}'.format(ave_recall))

    return ave_acc, ave_f1, ave_recall

In [19]:
c['lr'] = best_lr
c['decay'] = best_decay
mu = []
acc = []
f1 = []
recall = []
for r in range(1,11):
    c['mu']=r
    print('mu=',c['mu'])
    mu.append(c['mu'])
    ave_acc, ave_f1, ave_recall = cross_validation3(c)
    acc.append(ave_acc)
    f1.append(ave_f1)
    recall.append(ave_recall)
mu_list = list(zip(mu,acc,f1,recall))

mu= 1
Cross Validation

Fold 1:
The best result: 
Validation Accuracy: 0.9538910911746297, Macro F1-Score: 0.8075999480378455, Macro Recall: 0.9552788208412013

Fold 2:
The best result: 
Validation Accuracy: 0.9549942594718714, Macro F1-Score: 0.7813040578204546, Macro Recall: 0.9452652484244322

Fold 3:
The best result: 
Validation Accuracy: 0.9503646677471637, Macro F1-Score: 0.8268046590119359, Macro Recall: 0.9675550125324115

Fold 4:
The best result: 
Validation Accuracy: 0.951148830469882, Macro F1-Score: 0.8722825062525091, Macro Recall: 0.9924263235729828

Fold 5:
The best result: 
Validation Accuracy: 0.9689605881151726, Macro F1-Score: 0.8539020258003627, Macro Recall: 0.8968712858932114
accuracy of folds: [0.9538910911746297, 0.9549942594718714, 0.9503646677471637, 0.951148830469882, 0.9689605881151726]
average accuracy: 0.9558718873957439
f1 of folds: [0.8075999480378455, 0.7813040578204546, 0.8268046590119359, 0.8722825062525091, 0.8539020258003627]
average f1: 0.828378639

The best result: 
Validation Accuracy: 0.9796668829764222, Macro F1-Score: 0.9479298052971151, Macro Recall: 0.9666383121184986

Fold 3:
The best result: 
Validation Accuracy: 0.9812359779726698, Macro F1-Score: 0.8797830032630389, Macro Recall: 0.8887447849612194

Fold 4:
The best result: 
Validation Accuracy: 0.9828559027777778, Macro F1-Score: 0.8699661999827397, Macro Recall: 0.9628112278014669

Fold 5:
The best result: 
Validation Accuracy: 0.9874702959602506, Macro F1-Score: 0.8932006520021821, Macro Recall: 0.9378251591727126
accuracy of folds: [0.9811843929490989, 0.9796668829764222, 0.9812359779726698, 0.9828559027777778, 0.9874702959602506]
average accuracy: 0.9824826905272438
f1 of folds: [0.9060658018645444, 0.9479298052971151, 0.8797830032630389, 0.8699661999827397, 0.8932006520021821]
average f1: 0.899389092481924
recall of folds: [0.924371831491275, 0.9666383121184986, 0.8887447849612194, 0.9628112278014669, 0.9378251591727126]
average recall: 0.9360782631090345
mu= 9
Cr

In [20]:
mu_list_ = np.array(mu_list)
best_mu = mu_list_[np.argmax(mu_list_[:,3]),0]
print(best_mu)

1.0
